* original code from https://lsjsj92.tistory.com/569

In [1]:
from sklearn.decomposition import TruncatedSVD
from scipy.sparse.linalg import svds

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import ast
import warnings
warnings.filterwarnings("ignore")

from database import fetch_data_music_and_groups_extended, fetch_log_from_all_user, fetch_artist, fetch_bookmark, fetch_user, save_recom_musics

In [2]:
music_data = fetch_data_music_and_groups_extended()
column_names = ['music_id', 'title', 'genre', 'lyrics', 'release_date', 'likes', 'view',
                'highest_pitch', 'lowest_pitch', 'high_pitch_ratio', 'low_pitch_ratio',
                'pitch_average', 'pitch_stat', 'group_id', 'group_name', 'group_type']

music_data.columns = column_names

music_data['pitch_stat'] = music_data['pitch_stat'].apply(ast.literal_eval)

music_data

2024-10-25 10:33:40.526237: [SQL] SELECT m.music_id, m.title, m.genre, m.lyrics, m.release_date, m.likes, m.`view`, ma.highest_pitch, ma.lowest_pitch, ma.high_pitch_ratio, ma.low_pitch_ratio, ma.pitch_average, ma.pitch_stat, g.group_id, g.group_name, g.group_type FROM (music m NATURAL JOIN music_analysis ma) NATURAL JOIN groups g WHERE ma.`status` = 'COMPLETE'


,music_id,title,genre,lyrics,release_date,likes,view,highest_pitch,lowest_pitch,high_pitch_ratio,low_pitch_ratio,pitch_average,pitch_stat,group_id,group_name,group_type
0,169,야생화,BALLADE,하얗게 피어난 얼음 꽃 하나가\n달가운 바람에 얼굴을 내밀어\n아무 말 못했던 이름...,2014-03-28,2,42,926.378401,125.573964,0.199561,0.000000,271.857078,"{'C2': 0.0, 'D2': 0.0, 'E2': 0.0, 'F2': 0.0, '...",11,박효신,SOLO
1,170,"너에게 난, 나에게 넌",FOLK,너에게 난 해질녘 노을처럼\n한 편의 아름다운 추억이 되고\n소중했던 우리 푸르던 ...,2003-02-01,0,36,944.986819,140.076631,0.397028,0.000000,344.381940,"{'C2': 0.0, 'D2': 0.0, 'E2': 0.0, 'F2': 0.0, '...",13,자전거 탄 풍경,GROUP
2,171,너였다면,OST,왜 너에겐 그렇게 어려운지\n애를 쓰는 나를\n제대로 봐주는 게\n너 하나에 이토록...,2016-05-31,0,25,513.832400,145.657815,0.345133,0.000000,294.095022,"{'C2': 0.0, 'D2': 0.0, 'E2': 0.0, 'F2': 0.0, '...",12,마크툽 (MAKTUB),SOLO
3,172,시작,OST,새로운 시작은 늘 설레게 하지\n모든 걸 이겨낼 것처럼\n시간을 뒤쫓는 시계바늘처럼...,2020-02-01,0,8,428.380936,149.105775,0.331224,0.000000,300.080222,"{'C2': 0.0, 'D2': 0.0, 'E2': 0.0, 'F2': 0.0, '...",14,가호 (Gaho),SOLO
4,173,어땠을까 (Feat. 박정현),RAP,내가 그때 널 잡았더라면\n너와 나 지금보다 행복했을까\n마지막에 널\n안아줬다면 ...,2012-07-15,1,9,620.806132,123.754295,0.485465,0.000000,317.807284,"{'C2': 0.0, 'D2': 0.0, 'E2': 0.0, 'F2': 0.0, '...",15,싸이 (PSY),SOLO
5,174,흔들리는 꽃들 속에서 네 샴푸향이 느껴진거야,OST,흔들리는 꽃들 속에서\n네 샴푸향이 느껴진거야\n스쳐지나간건가 뒤돌아보지만\n그냥 ...,2019-08-23,0,7,500.865280,104.255744,0.052632,0.005848,245.927647,"{'C2': 0.0, 'D2': 0.0, 'E2': 0.0, 'F2': 0.0, '...",16,장범준,SOLO
6,175,Don't Cry,BALLADE,그대의 눈물이 마를 때까지\n우리의 사랑을 볼 수 없을테니\n울지 말아요 이제야\n...,2003-08-01,0,4,796.514962,124.552403,0.440945,0.000000,321.024447,"{'C2': 0.0, 'D2': 0.0, 'E2': 0.0, 'F2': 0.0, '...",17,더 크로스,GROUP
7,176,잠시만 안녕,BALLADE,행복을 줄 수 없었어\n그런데 사랑을 했어\n니곁에 감히 머무른\n내 욕심을 용서치...,2007-05-10,0,12,885.293572,166.179307,0.493438,0.000000,360.619057,"{'C2': 0.0, 'D2': 0.0, 'E2': 0.0, 'F2': 0.0, '...",18,엠씨더맥스 (M.C the MAX),SOLO
8,177,인형의 꿈,OST,그대 먼 곳만 보네요\n내가 바로 여기 있는데\n조금만 고개를 돌려도\n날 볼 수 ...,2003-09-09,1,5,588.248207,232.075376,0.021583,0.120504,386.938462,"{'C2': 0.0, 'D2': 0.0, 'E2': 0.0, 'F2': 0.0, '...",19,러브홀릭,SOLO
9,178,외톨이야,ROCK,외톨이야 외톨이야\n\n외톨이야 외톨이야\n\n\n\n봐봐 나를 봐봐 똑바로 내 두...,2010-01-14,0,4,557.989885,150.122581,0.468354,0.000000,339.719083,"{'C2': 0.0, 'D2': 0.0, 'E2': 0.0, 'F2': 0.0, '...",20,CNBLUE (씨엔블루),GROUP


In [3]:
artists = fetch_artist()
column_names = ['group_id', 'artist_id', 'artist_name', 'gender', 'nation']
artists.columns = column_names

artists

2024-10-25 10:33:40.600336: [SQL] SELECT gai.group_id, a.artist_id, a.artist_name, a.gender, a.nation FROM (SELECT g.group_id, gm.artist_id FROM groups g NATURAL JOIN group_member gm) gai    INNER JOIN artist a ON gai.artist_id = a.artist_id


,group_id,artist_id,artist_name,gender,nation
0,7,2,이영지,FEMALE,대한민국
1,11,11,박효신,MALE,대한민국
2,13,13,강인봉,MALE,대한민국
3,13,14,김형섭,MALE,대한민국
4,13,15,송봉주,MALE,대한민국
...,...,...,...,...,...
102,74,148,Roger Taylor,MALE,영국
103,74,149,Freddie Mercury,MALE,영국
104,74,150,John Deacon,MALE,영국
105,75,151,지코 (ZICO),MALE,대한민국


In [4]:
artist_dict = {}

for at in artists.itertuples():
    if at.group_id in artist_dict:
        artist_dict[at.group_id].append(at)
    else:
        artist_dict[at.group_id] = [at]

In [5]:
def classify_gender(gender_list):
    # 'OTHER'이 하나라도 있으면 '기타'
    if 'OTHER' in gender_list:
        return 'OTHER'
    
    # 'FEMALE'과 'MALE'이 섞여 있으면 '혼성'
    if 'FEMALE' in gender_list and 'MALE' in gender_list:
        return 'MIX'
    
    # 'FEMALE'만 있으면 '여성'
    if all(g == 'FEMALE' for g in gender_list):
        return 'FEMALE'
    
    # 'MALE'만 있으면 '남성'
    if all(g == 'MALE' for g in gender_list):
        return 'MALE'
    
    # 예외 처리를 위해 기본 반환값 (조건에 맞지 않는 경우)
    return 'OTHER'

# 노래 성별(남자, 여자, 혼성, 그외) 구분 추가
music_data['gender'] = music_data.apply(lambda row: classify_gender([at.gender for at in artist_dict[row.group_id]]), axis=1)

In [6]:
log_data = fetch_log_from_all_user()
column_names = ['event_id', 'created_at', 'event', 'target_id', 'user_id']

log_data.columns = column_names

log_data.head()

2024-10-25 10:33:40.647657: [SQL] SELECT * FROM log WHERE created_at >= DATE_SUB(NOW(), INTERVAL 3 MONTH)


,event_id,created_at,event,target_id,user_id
0,1,2024-10-20 22:59:18.581643,viewMusicDetail,383,1
1,2,2024-10-20 22:59:28.355868,viewMusicDetail,180,1
2,3,2024-10-20 22:59:33.554950,viewMusicDetail,176,1
3,4,2024-10-21 11:20:40.244589,viewMusicDetail,192,1
4,5,2024-10-21 11:20:43.759666,viewMusicDetail,203,1


In [7]:
log_data_test = log_data
log_data_test = log_data_test.drop('created_at', axis=1)
log_data_test.head()

,event_id,event,target_id,user_id
0,1,viewMusicDetail,383,1
1,2,viewMusicDetail,180,1
2,3,viewMusicDetail,176,1
3,4,viewMusicDetail,192,1
4,5,viewMusicDetail,203,1


In [8]:
import threading
from concurrent.futures import ThreadPoolExecutor

# 특정 key에 대한 lock을 관리하는 클래스
class KeyLockDict:
    def __init__(self):
        self.data = {}
        self.locks = {}
        self.global_lock = threading.Lock()

    def _get_lock(self, key):
        # 특정 key에 대한 lock을 생성 또는 반환
        with self.global_lock:
            if key not in self.locks:
                self.locks[key] = threading.Lock()
            return self.locks[key]

    def set(self, key, value):
        lock = self._get_lock(key)
        with lock:
            self.data[key] = value

    def get(self, key):
        lock = self._get_lock(key)
        with lock:
            return self.data.get(key)
        
    def add(self, key, value):
        lock = self._get_lock(key)
        with lock:
            if self.data.get(key) is not None:
                self.data[key] += value
            else:
                self.data[key] = 3 + value

    def remove(self, key):
        lock = self._get_lock(key)
        with lock:
            if key in self.data:
                del self.data[key]
                del self.locks[key]
                
# KeyLockDict 생성
shared_dict = KeyLockDict()

# 데이터 처리 함수 (Thread-safe)
def process_row(row):
    key = (row['target_id'], row['user_id'])
    event = row['event']

    if event == 'viewMusicDetail':
        shared_dict.add(key, 0.1)
    elif event == 'closeMusicDetail':
        shared_dict.add(key, -0.5)
    elif event == 'bookmarkMusic':
        shared_dict.add(key, 1)
    elif event == 'unbookmarkMusic':
        shared_dict.add(key, -1)
    elif event == 'feedbackPositive':
        shared_dict.add(key, 1.5)
    elif event == 'feedbackNegative':
        shared_dict.add(key, -2)

# ThreadPoolExecutor를 사용한 병렬 처리
with ThreadPoolExecutor() as executor:
    executor.map(process_row, [row for _, row in log_data.iterrows()])

# 결과 출력
print("Final shared_dict data:", shared_dict.data)

rating_data = pd.DataFrame(list(shared_dict.data.items()), columns=['Key', 'rating'])

rating_data[['music_id', 'user_id']] = pd.DataFrame(rating_data['Key'].tolist(), index=rating_data.index)
rating_data = rating_data.drop('Key', axis=1)

# 열 순서 정렬
rating_data = rating_data[['music_id', 'user_id', 'rating']]

Final shared_dict data: {(383, 1): 3.2, (180, 1): 3.1, (176, 1): 3.2, (192, 1): 3.1, (203, 1): 5.699999999999999, (201, 1): 4.499999999999998, (386, 1): 3.2, (169, 1): 3.1, (205, 1): 1.1, (204, 1): 3.1, (170, 1): 3.1, (192, 2): 4.1, (205, 2): 4.1, (177, 2): 4.1, (384, 1): 3.1, (191, 1): 3.1, (206, 1): 1, (395, 1): 3.2, (393, 1): 3.1, (396, 1): 3.1, (182, 1): 3.1, (387, 1): 3.1, (397, 1): 3.1, (386, 8): 3.1, (203, 5): 4.5, (192, 6): 3.1, (393, 21): 3.1, (169, 19): 4}


In [9]:
rating_data.head()

,music_id,user_id,rating
0,383,1,3.2
1,180,1,3.1
2,176,1,3.2
3,192,1,3.1
4,203,1,5.7


In [10]:
music_data_test = music_data
music_data_test.drop(['genre', 'lyrics', 'release_date', 'likes', 'view', 'highest_pitch', 'lowest_pitch', 'high_pitch_ratio', 'low_pitch_ratio',
                'pitch_average', 'pitch_stat', 'group_id', 'group_name', 'group_type'], axis = 1)
music_data_test.head()

,music_id,title,genre,lyrics,release_date,likes,view,highest_pitch,lowest_pitch,high_pitch_ratio,low_pitch_ratio,pitch_average,pitch_stat,group_id,group_name,group_type,gender
0,169,야생화,BALLADE,하얗게 피어난 얼음 꽃 하나가\n달가운 바람에 얼굴을 내밀어\n아무 말 못했던 이름...,2014-03-28,2,42,926.378401,125.573964,0.199561,0.0,271.857078,"{'C2': 0.0, 'D2': 0.0, 'E2': 0.0, 'F2': 0.0, '...",11,박효신,SOLO,MALE
1,170,"너에게 난, 나에게 넌",FOLK,너에게 난 해질녘 노을처럼\n한 편의 아름다운 추억이 되고\n소중했던 우리 푸르던 ...,2003-02-01,0,36,944.986819,140.076631,0.397028,0.0,344.381940,"{'C2': 0.0, 'D2': 0.0, 'E2': 0.0, 'F2': 0.0, '...",13,자전거 탄 풍경,GROUP,MALE
2,171,너였다면,OST,왜 너에겐 그렇게 어려운지\n애를 쓰는 나를\n제대로 봐주는 게\n너 하나에 이토록...,2016-05-31,0,25,513.832400,145.657815,0.345133,0.0,294.095022,"{'C2': 0.0, 'D2': 0.0, 'E2': 0.0, 'F2': 0.0, '...",12,마크툽 (MAKTUB),SOLO,MALE
3,172,시작,OST,새로운 시작은 늘 설레게 하지\n모든 걸 이겨낼 것처럼\n시간을 뒤쫓는 시계바늘처럼...,2020-02-01,0,8,428.380936,149.105775,0.331224,0.0,300.080222,"{'C2': 0.0, 'D2': 0.0, 'E2': 0.0, 'F2': 0.0, '...",14,가호 (Gaho),SOLO,MALE
4,173,어땠을까 (Feat. 박정현),RAP,내가 그때 널 잡았더라면\n너와 나 지금보다 행복했을까\n마지막에 널\n안아줬다면 ...,2012-07-15,1,9,620.806132,123.754295,0.485465,0.0,317.807284,"{'C2': 0.0, 'D2': 0.0, 'E2': 0.0, 'F2': 0.0, '...",15,싸이 (PSY),SOLO,MALE


In [11]:
music_rating_data = pd.merge(rating_data, music_data, on ='music_id')
music_rating_data.head()

,music_id,user_id,rating,title,genre,lyrics,release_date,likes,view,highest_pitch,lowest_pitch,high_pitch_ratio,low_pitch_ratio,pitch_average,pitch_stat,group_id,group_name,group_type,gender
0,383,1,3.2,다행이다,BALLADE,그대를 만나고\n그대의 머릿결을 만질 수가 있어서\n그대를 만나고\n그대와 마주보며...,2007-04-19,0,22,378.239188,125.874867,0.077957,0.000000,261.205990,"{'C2': 0.0, 'D2': 0.0, 'E2': 0.0, 'F2': 0.0, '...",61,이적,SOLO,MALE
1,180,1,3.1,취중진담,BALLADE,그래 난 취했는지도 몰라\n실수인지도 몰라\n아침이면 까마득히 생각이 안나\n불안해...,2007-03-15,0,8,688.456088,92.717461,0.236074,0.002653,254.914448,"{'C2': 0.0, 'D2': 0.0, 'E2': 0.0, 'F2': 0.0, '...",22,김동률,SOLO,MALE
2,176,1,3.2,잠시만 안녕,BALLADE,행복을 줄 수 없었어\n그런데 사랑을 했어\n니곁에 감히 머무른\n내 욕심을 용서치...,2007-05-10,0,12,885.293572,166.179307,0.493438,0.000000,360.619057,"{'C2': 0.0, 'D2': 0.0, 'E2': 0.0, 'F2': 0.0, '...",18,엠씨더맥스 (M.C the MAX),SOLO,MALE
3,192,1,3.1,Spicy,DANCE,Uh Uh Too Spicy\nYou want my A to the Z\nBut y...,2023-05-08,1,11,703.966140,193.677050,0.074554,0.141005,387.823861,"{'C2': 0.0, 'D2': 0.0, 'E2': 0.0, 'F2': 0.0, '...",43,aespa,GROUP,FEMALE
4,203,1,5.7,주저하는 연인들을 위해,INDIE,나는 읽기 쉬운 마음이야\n당신도 스윽 훑고 가셔요\n달랠 길 없는 외로운 마음 있...,2019-03-13,1,11,465.799037,131.582028,0.175559,0.000000,272.035502,"{'C2': 0.0, 'D2': 0.0, 'E2': 0.0, 'F2': 0.0, '...",54,잔나비,GROUP,MALE


In [12]:
music_rating_data.shape

(28, 19)

In [13]:
user_music_rating = music_rating_data.pivot_table('rating', index = 'user_id', columns='title').fillna(3)

In [14]:
user_music_rating.shape

(7, 21)

In [15]:
user_music_rating.head()

title,"Beauty and the Beast (from ""Beauty and the Beast"")",Boys And Girls (Feat. Babylon),Don't Stop Me Now,Let's Get Loud,My All,Spicy,가지마 가지마,"너에게 난, 나에게 넌",다행이다,부동의 첫사랑,...,소주 한 잔,야생화,오늘도 빛나는 너에게,인형의 꿈,잠시만 안녕,주저하는 연인들을 위해,첫사랑,취중진담,친구 얘긴데 (feat. Skinny Brown),퀸카 (Queencard)
user_id,,,,,,,,,,,,,,,,,,,,,
1,3.2,3.1,3.1,3.1,3.1,3.1,3.1,3.1,3.2,4.5,...,3.1,3.1,1.0,3.0,3.2,5.7,1.1,3.1,3.1,3.1
2,3.0,3.0,3.0,3.0,3.0,4.1,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,4.1,3.0,3.0,4.1,3.0,3.0,3.0
5,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,4.5,3.0,3.0,3.0,3.0
6,3.0,3.0,3.0,3.0,3.0,3.1,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
8,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0


In [16]:
user_music_rating

title,"Beauty and the Beast (from ""Beauty and the Beast"")",Boys And Girls (Feat. Babylon),Don't Stop Me Now,Let's Get Loud,My All,Spicy,가지마 가지마,"너에게 난, 나에게 넌",다행이다,부동의 첫사랑,...,소주 한 잔,야생화,오늘도 빛나는 너에게,인형의 꿈,잠시만 안녕,주저하는 연인들을 위해,첫사랑,취중진담,친구 얘긴데 (feat. Skinny Brown),퀸카 (Queencard)
user_id,,,,,,,,,,,,,,,,,,,,,
1,3.2,3.1,3.1,3.1,3.1,3.1,3.1,3.1,3.2,4.5,...,3.1,3.1,1.0,3.0,3.2,5.7,1.1,3.1,3.1,3.1
2,3.0,3.0,3.0,3.0,3.0,4.1,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,4.1,3.0,3.0,4.1,3.0,3.0,3.0
5,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,4.5,3.0,3.0,3.0,3.0
6,3.0,3.0,3.0,3.0,3.0,3.1,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
8,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
19,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,4.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
21,3.0,3.0,3.0,3.0,3.1,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0


In [17]:
# 나중에 user_id를 이용해 index로 접근하기 위해 저장
user_index_dict = dict()
for idx, uid in enumerate(user_music_rating.index):
    user_index_dict[uid] = idx

user_index_dict

{1: 0, 2: 1, 5: 2, 6: 3, 8: 4, 19: 5, 21: 6}

In [18]:
# 로그에 나타나지 않는 음악 추가
all_music_title = music_data_test['title'].tolist()
missing_titles = [title for title in all_music_title if title not in user_music_rating.columns.tolist()]

user_music_rating = user_music_rating.reindex(
    index=list(user_music_rating.index),
    columns=list(user_music_rating.columns) + missing_titles,
    fill_value=3
)
user_music_rating.head()

title,"Beauty and the Beast (from ""Beauty and the Beast"")",Boys And Girls (Feat. Babylon),Don't Stop Me Now,Let's Get Loud,My All,Spicy,가지마 가지마,"너에게 난, 나에게 넌",다행이다,부동의 첫사랑,...,"모든 날, 모든 순간",FRIEND THE END,바람기억,Everytime,She's Gone,Because Of You,Superstar,What's Up?,Listen,검은 행복
user_id,,,,,,,,,,,,,,,,,,,,,
1,3.2,3.1,3.1,3.1,3.1,3.1,3.1,3.1,3.2,4.5,...,3,3,3,3,3,3,3,3,3,3
2,3.0,3.0,3.0,3.0,3.0,4.1,3.0,3.0,3.0,3.0,...,3,3,3,3,3,3,3,3,3,3
5,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3,3,3,3,3,3,3,3,3,3
6,3.0,3.0,3.0,3.0,3.0,3.1,3.0,3.0,3.0,3.0,...,3,3,3,3,3,3,3,3,3,3
8,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3,3,3,3,3,3,3,3,3,3


In [19]:
# matrix는 pivot_table 값을 numpy matrix로 만든 것 
matrix = user_music_rating.values

# user_ratings_mean은 사용자의 평균 평점 
user_ratings_mean = np.mean(matrix, axis = 1)

# R_user_mean : 사용자-음악에 대해 사용자 평균 평점을 뺀 것.
matrix_user_mean = matrix - user_ratings_mean.reshape(-1, 1)

In [20]:
matrix_user_mean

array([[ 1.57407407e-01,  5.74074074e-02,  5.74074074e-02,
         5.74074074e-02,  5.74074074e-02,  5.74074074e-02,
         5.74074074e-02,  5.74074074e-02,  1.57407407e-01,
         1.45740741e+00,  1.57407407e-01,  5.74074074e-02,
         5.74074074e-02, -2.04259259e+00, -4.25925926e-02,
         1.57407407e-01,  2.65740741e+00, -1.94259259e+00,
         5.74074074e-02,  5.74074074e-02,  5.74074074e-02,
        -4.25925926e-02, -4.25925926e-02, -4.25925926e-02,
        -4.25925926e-02, -4.25925926e-02, -4.25925926e-02,
        -4.25925926e-02, -4.25925926e-02, -4.25925926e-02,
        -4.25925926e-02, -4.25925926e-02, -4.25925926e-02,
        -4.25925926e-02, -4.25925926e-02, -4.25925926e-02,
        -4.25925926e-02, -4.25925926e-02, -4.25925926e-02,
        -4.25925926e-02, -4.25925926e-02, -4.25925926e-02,
        -4.25925926e-02, -4.25925926e-02, -4.25925926e-02,
        -4.25925926e-02, -4.25925926e-02, -4.25925926e-02,
        -4.25925926e-02, -4.25925926e-02, -4.25925926e-0

In [21]:
pd.DataFrame(matrix_user_mean, columns = user_music_rating.columns).head()

title,"Beauty and the Beast (from ""Beauty and the Beast"")",Boys And Girls (Feat. Babylon),Don't Stop Me Now,Let's Get Loud,My All,Spicy,가지마 가지마,"너에게 난, 나에게 넌",다행이다,부동의 첫사랑,...,"모든 날, 모든 순간",FRIEND THE END,바람기억,Everytime,She's Gone,Because Of You,Superstar,What's Up?,Listen,검은 행복
0,0.157407,0.057407,0.057407,0.057407,0.057407,0.057407,0.057407,0.057407,0.157407,1.457407,...,-0.042593,-0.042593,-0.042593,-0.042593,-0.042593,-0.042593,-0.042593,-0.042593,-0.042593,-0.042593
1,-0.061111,-0.061111,-0.061111,-0.061111,-0.061111,1.038889,-0.061111,-0.061111,-0.061111,-0.061111,...,-0.061111,-0.061111,-0.061111,-0.061111,-0.061111,-0.061111,-0.061111,-0.061111,-0.061111,-0.061111
2,-0.027778,-0.027778,-0.027778,-0.027778,-0.027778,-0.027778,-0.027778,-0.027778,-0.027778,-0.027778,...,-0.027778,-0.027778,-0.027778,-0.027778,-0.027778,-0.027778,-0.027778,-0.027778,-0.027778,-0.027778
3,-0.001852,-0.001852,-0.001852,-0.001852,-0.001852,0.098148,-0.001852,-0.001852,-0.001852,-0.001852,...,-0.001852,-0.001852,-0.001852,-0.001852,-0.001852,-0.001852,-0.001852,-0.001852,-0.001852,-0.001852
4,-0.001852,-0.001852,-0.001852,-0.001852,-0.001852,-0.001852,-0.001852,-0.001852,-0.001852,-0.001852,...,-0.001852,-0.001852,-0.001852,-0.001852,-0.001852,-0.001852,-0.001852,-0.001852,-0.001852,-0.001852


In [22]:
U, sigma, Vt = svds(matrix_user_mean, k = 6)

In [23]:
print(U.shape)
print(sigma.shape)
print(Vt.shape)

(7, 6)
(6,)
(6, 54)


In [24]:
# 현재 이 Sigma 행렬은 0이 아닌 값만 1차원 행렬로 표현된 상태입니다.
# 즉, 0이 포함된 대칭행렬로 변환할 때는 numpy의 diag를 이용해야 합니다.

sigma = np.diag(sigma)
sigma.shape

(6, 6)

이렇게 대칭 행렬로 변환이 되었습니다.

현재 까지 상황을 정리하면 아래와 같습니다.

원본 user-music 평점 행렬이 있었음
이를 user의 평균 점수를 빼서 matrix_user_mean 이라는 행렬로 만듬
2번의 값을 SVD를 적용해 U, Sigma, Vt 행렬을 구했음
Sigma 행렬은 현재 0이 포함이 되지 않은 값으로만 구성되어 있음. 이를 대칭행렬로 변환
자! 이제 여기서 matrix_user_mean을 SVD를 적용해 분해를 한 상태입니다.
이제, 다시 원본 행렬로 복구시켜야겠죠?

원본 행렬로 복구시키는 방법은 아래와 같습니다.

U, Sigma, Vt의 내적을 수행
즉, np.dot(np.dot(U, sigma), Vt)를 수행하면 됩니다.

그리고 아까 사용자 평균을 빼주었으니 여기서는 더해줍니다.

In [25]:
# U, Sigma, Vt의 내적을 수행하면, 다시 원본 행렬로 복원이 된다. 
# 거기에 + 사용자 평균 rating을 적용한다. 
svd_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)

In [26]:
df_svd_preds = pd.DataFrame(svd_user_predicted_ratings, columns = user_music_rating.columns)
df_svd_preds.head()

title,"Beauty and the Beast (from ""Beauty and the Beast"")",Boys And Girls (Feat. Babylon),Don't Stop Me Now,Let's Get Loud,My All,Spicy,가지마 가지마,"너에게 난, 나에게 넌",다행이다,부동의 첫사랑,...,"모든 날, 모든 순간",FRIEND THE END,바람기억,Everytime,She's Gone,Because Of You,Superstar,What's Up?,Listen,검은 행복
0,3.199988,3.099995,3.099995,3.099995,3.100009,3.100480,3.099995,3.099995,3.199988,4.499908,...,3.000001,3.000001,3.000001,3.000001,3.000001,3.000001,3.000001,3.000001,3.000001,3.000001
1,2.999949,2.999976,2.999976,2.999976,3.000038,4.102116,2.999976,2.999976,2.999949,2.999594,...,3.000003,3.000003,3.000003,3.000003,3.000003,3.000003,3.000003,3.000003,3.000003,3.000003
2,3.000021,3.000010,3.000010,3.000010,2.999985,2.999137,3.000010,3.000010,3.000021,3.000166,...,2.999999,2.999999,2.999999,2.999999,2.999999,2.999999,2.999999,2.999999,2.999999,2.999999
3,3.001475,3.000688,3.000688,3.000688,2.998920,3.039116,3.000688,3.000688,3.001475,3.011696,...,2.999902,2.999902,2.999902,2.999902,2.999902,2.999902,2.999902,2.999902,2.999902,2.999902
4,3.000056,3.000026,3.000026,3.000026,2.999959,2.997686,3.000026,3.000026,3.000056,3.000445,...,2.999996,2.999996,2.999996,2.999996,2.999996,2.999996,2.999996,2.999996,2.999996,2.999996


자! 이제 함수를 하나 만듭니다. 이 함수의 기능은 아래와 같습니다.

- 인자로 사용자 아이디, 음악 정보 테이블, 평점 테이블 등을 받음
- 사용자 아이디에 SVD로 나온 결과의 음악 평점이 가장 높은 데이터 순으로 정렬
- 사용자가 북마크한 데이터를 제외

In [27]:
df_svd_preds

title,"Beauty and the Beast (from ""Beauty and the Beast"")",Boys And Girls (Feat. Babylon),Don't Stop Me Now,Let's Get Loud,My All,Spicy,가지마 가지마,"너에게 난, 나에게 넌",다행이다,부동의 첫사랑,...,"모든 날, 모든 순간",FRIEND THE END,바람기억,Everytime,She's Gone,Because Of You,Superstar,What's Up?,Listen,검은 행복
0,3.199988,3.099995,3.099995,3.099995,3.100009,3.100480,3.099995,3.099995,3.199988,4.499908,...,3.000001,3.000001,3.000001,3.000001,3.000001,3.000001,3.000001,3.000001,3.000001,3.000001
1,2.999949,2.999976,2.999976,2.999976,3.000038,4.102116,2.999976,2.999976,2.999949,2.999594,...,3.000003,3.000003,3.000003,3.000003,3.000003,3.000003,3.000003,3.000003,3.000003,3.000003
2,3.000021,3.000010,3.000010,3.000010,2.999985,2.999137,3.000010,3.000010,3.000021,3.000166,...,2.999999,2.999999,2.999999,2.999999,2.999999,2.999999,2.999999,2.999999,2.999999,2.999999
3,3.001475,3.000688,3.000688,3.000688,2.998920,3.039116,3.000688,3.000688,3.001475,3.011696,...,2.999902,2.999902,2.999902,2.999902,2.999902,2.999902,2.999902,2.999902,2.999902,2.999902
4,3.000056,3.000026,3.000026,3.000026,2.999959,2.997686,3.000026,3.000026,3.000056,3.000445,...,2.999996,2.999996,2.999996,2.999996,2.999996,2.999996,2.999996,2.999996,2.999996,2.999996
5,3.000001,3.000000,3.000000,3.000000,2.999999,2.999958,3.000000,3.000000,3.000001,3.000008,...,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000
6,3.000026,3.000012,3.000012,3.000012,3.099981,2.998920,3.000012,3.000012,3.000026,3.000208,...,2.999998,2.999998,2.999998,2.999998,2.999998,2.999998,2.999998,2.999998,2.999998,2.999998


In [28]:
# 1번 사용자가 북마크한 music_id 조회
bookmark_data = fetch_bookmark(1)

column_names = ['music_id', 'title']
bookmark_data.columns = column_names

bookmark_data

2024-10-25 10:33:41.117861: [SQL] SELECT music_id, title FROM bookmark NATURAL JOIN music WHERE user_id = 1


,music_id,title
0,203,주저하는 연인들을 위해
1,201,부동의 첫사랑
2,169,야생화


In [29]:
# 최종적으로 만든 df_svd_preds에서 사용자 index에 따라 데이터 정렬 -> 예상 평점이 높은 순으로 정렬
sorted_user_predictions = df_svd_preds.iloc[user_index_dict[1]]#.sort_values(ascending=False)
sorted_user_predictions

title
Beauty and the Beast (from "Beauty and the Beast")    3.199988
Boys And Girls (Feat. Babylon)                        3.099995
Don't Stop Me Now                                     3.099995
Let's Get Loud                                        3.099995
My All                                                3.100009
Spicy                                                 3.100480
가지마 가지마                                               3.099995
너에게 난, 나에게 넌                                          3.099995
다행이다                                                  3.199988
부동의 첫사랑                                               4.499908
사랑은 은하수 다방에서                                          3.200018
소주 한 잔                                                3.099995
야생화                                                   3.100000
오늘도 빛나는 너에게                                           1.000125
인형의 꿈                                                 2.999700
잠시만 안녕                                           

In [30]:
# 사용자가 이미 북마크한 곡은 제거
recommendations = sorted_user_predictions.drop(bookmark_data.title.tolist())
recommendations

title
Beauty and the Beast (from "Beauty and the Beast")    3.199988
Boys And Girls (Feat. Babylon)                        3.099995
Don't Stop Me Now                                     3.099995
Let's Get Loud                                        3.099995
My All                                                3.100009
Spicy                                                 3.100480
가지마 가지마                                               3.099995
너에게 난, 나에게 넌                                          3.099995
다행이다                                                  3.199988
사랑은 은하수 다방에서                                          3.200018
소주 한 잔                                                3.099995
오늘도 빛나는 너에게                                           1.000125
인형의 꿈                                                 2.999700
잠시만 안녕                                                3.199988
첫사랑                                                   1.099818
취중진담                                             

In [31]:
recommendations_df = pd.DataFrame(recommendations)
recommendations_df.columns = ['pred']

In [32]:
recommendations_data = recommendations_df.merge(music_data, on='title')
recommendations_data.head()

,title,pred,music_id,genre,lyrics,release_date,likes,view,highest_pitch,lowest_pitch,high_pitch_ratio,low_pitch_ratio,pitch_average,pitch_stat,group_id,group_name,group_type,gender
0,"Beauty and the Beast (from ""Beauty and the Bea...",3.199988,395,OST,None,1992-03-30,0,2,882.665004,170.163455,0.669540,0.000000,404.741144,"{'C2': 0.0, 'D2': 0.0, 'E2': 0.0, 'F2': 0.0, '...",72,"Peabo Bryson, Celine Dion",GROUP,MIX
1,Boys And Girls (Feat. Babylon),3.099995,397,RAP,Don't worry about money\n놀자 오빠가 무리할게\n넌 그냥 옆에서...,2015-11-03,0,1,469.292125,123.944968,0.123616,0.000000,270.923070,"{'C2': 0.0, 'D2': 0.0, 'E2': 0.0, 'F2': 0.0, '...",75,지코 (ZICO),SOLO,MALE
2,Don't Stop Me Now,3.099995,396,ROCK,"Tonight, I'm gonna have myself a real good tim...",1976-01-26,0,1,507.861756,164.907098,0.505030,0.000000,319.656593,"{'C2': 0.0, 'D2': 0.0, 'E2': 0.0, 'F2': 0.0, '...",74,Queen,GROUP,MALE
3,Let's Get Loud,3.099995,387,POP,let's get loud let's get loud\nturn the music ...,1999-06-29,0,16,597.670991,205.814429,0.039370,0.226378,346.274942,"{'C2': 0.0, 'D2': 0.0, 'E2': 0.0, 'F2': 0.0, '...",64,Jennifer Lopez,SOLO,FEMALE
4,My All,3.100009,393,RNB,I am thinking of you\n당신을 생각해요\nin my sleeples...,1997-09-16,0,1,694.707279,144.864262,0.173242,0.418525,371.087818,"{'C2': 0.0, 'D2': 0.0, 'E2': 0.0, 'F2': 0.0, '...",69,Mariah Carey,SOLO,FEMALE


# 사용자의 선호 장르와 나이대에 따른 가산점 계산

In [33]:
import pymysql

user_data = fetch_user(1)
print(user_data)

{'age': 20, 'gender': 'MALE', 'genre': ['BALLADE', 'DANCE', 'RAP'], 'highest_pitch': 351.0, 'lowest_pitch': 131.0}


In [34]:
recommendations_data.head()

,title,pred,music_id,genre,lyrics,release_date,likes,view,highest_pitch,lowest_pitch,high_pitch_ratio,low_pitch_ratio,pitch_average,pitch_stat,group_id,group_name,group_type,gender
0,"Beauty and the Beast (from ""Beauty and the Bea...",3.199988,395,OST,None,1992-03-30,0,2,882.665004,170.163455,0.669540,0.000000,404.741144,"{'C2': 0.0, 'D2': 0.0, 'E2': 0.0, 'F2': 0.0, '...",72,"Peabo Bryson, Celine Dion",GROUP,MIX
1,Boys And Girls (Feat. Babylon),3.099995,397,RAP,Don't worry about money\n놀자 오빠가 무리할게\n넌 그냥 옆에서...,2015-11-03,0,1,469.292125,123.944968,0.123616,0.000000,270.923070,"{'C2': 0.0, 'D2': 0.0, 'E2': 0.0, 'F2': 0.0, '...",75,지코 (ZICO),SOLO,MALE
2,Don't Stop Me Now,3.099995,396,ROCK,"Tonight, I'm gonna have myself a real good tim...",1976-01-26,0,1,507.861756,164.907098,0.505030,0.000000,319.656593,"{'C2': 0.0, 'D2': 0.0, 'E2': 0.0, 'F2': 0.0, '...",74,Queen,GROUP,MALE
3,Let's Get Loud,3.099995,387,POP,let's get loud let's get loud\nturn the music ...,1999-06-29,0,16,597.670991,205.814429,0.039370,0.226378,346.274942,"{'C2': 0.0, 'D2': 0.0, 'E2': 0.0, 'F2': 0.0, '...",64,Jennifer Lopez,SOLO,FEMALE
4,My All,3.100009,393,RNB,I am thinking of you\n당신을 생각해요\nin my sleeples...,1997-09-16,0,1,694.707279,144.864262,0.173242,0.418525,371.087818,"{'C2': 0.0, 'D2': 0.0, 'E2': 0.0, 'F2': 0.0, '...",69,Mariah Carey,SOLO,FEMALE


In [35]:
def update_rating_by_user_data(row, user_data):

    # 나이
    age_prefix = user_data['age'] // 10
    release = row['release_date'].year
    if age_prefix == 1 and 2018 <= release:
        row['pred'] += 0.3
    elif age_prefix == 2 and 2010 <= release <= 2024:
        row['pred'] += 0.3
    elif age_prefix == 3 and 1995 <= release <= 2015:
        row['pred'] += 0.3
    elif age_prefix == 4 and 1985 <= release <= 2005:
        row['pred'] += 0.3
    elif age_prefix == 5 and 1975 <= release <= 1995:
        row['pred'] += 0.3
    elif age_prefix == 6 and 1965 <= release <= 1985:
        row['pred'] += 0.3
    
    # 성별
    if row['gender'] == user_data['gender']:
        row['pred'] += 0.4
    
    # 장르
    if row['genre'] in user_data['genre']:
        row['pred'] += 0.4
        
    return row
    

recommendations_data = recommendations_data.apply(update_rating_by_user_data, axis=1, user_data=user_data)

In [36]:
recommendations_data.head()

,title,pred,music_id,genre,lyrics,release_date,likes,view,highest_pitch,lowest_pitch,high_pitch_ratio,low_pitch_ratio,pitch_average,pitch_stat,group_id,group_name,group_type,gender
0,"Beauty and the Beast (from ""Beauty and the Bea...",3.199988,395,OST,None,1992-03-30,0,2,882.665004,170.163455,0.669540,0.000000,404.741144,"{'C2': 0.0, 'D2': 0.0, 'E2': 0.0, 'F2': 0.0, '...",72,"Peabo Bryson, Celine Dion",GROUP,MIX
1,Boys And Girls (Feat. Babylon),4.199995,397,RAP,Don't worry about money\n놀자 오빠가 무리할게\n넌 그냥 옆에서...,2015-11-03,0,1,469.292125,123.944968,0.123616,0.000000,270.923070,"{'C2': 0.0, 'D2': 0.0, 'E2': 0.0, 'F2': 0.0, '...",75,지코 (ZICO),SOLO,MALE
2,Don't Stop Me Now,3.499995,396,ROCK,"Tonight, I'm gonna have myself a real good tim...",1976-01-26,0,1,507.861756,164.907098,0.505030,0.000000,319.656593,"{'C2': 0.0, 'D2': 0.0, 'E2': 0.0, 'F2': 0.0, '...",74,Queen,GROUP,MALE
3,Let's Get Loud,3.099995,387,POP,let's get loud let's get loud\nturn the music ...,1999-06-29,0,16,597.670991,205.814429,0.039370,0.226378,346.274942,"{'C2': 0.0, 'D2': 0.0, 'E2': 0.0, 'F2': 0.0, '...",64,Jennifer Lopez,SOLO,FEMALE
4,My All,3.100009,393,RNB,I am thinking of you\n당신을 생각해요\nin my sleeples...,1997-09-16,0,1,694.707279,144.864262,0.173242,0.418525,371.087818,"{'C2': 0.0, 'D2': 0.0, 'E2': 0.0, 'F2': 0.0, '...",69,Mariah Carey,SOLO,FEMALE


# 음역대에 따른 가산점 계산

In [37]:
import bisect

class PitchConverter:
    freq_level = [
        65.41, 65.41, 73.42, 82.41, 87.31, 98.00, 110.00, 123.47, 130.81, 146.83, 164.81, 174.61,
        196.00, 220.00, 246.94, 261.63, 293.66, 329.63, 349.23, 392.00, 440.00, 493.88, 523.25,
        587.33, 659.25, 698.46, 783.99, 880.00, 987.77, 1046.50, 1174.66, 1174.66
    ]

    pitch_string = [
        "C2", "C2", "D2", "E2", "F2", "G2", "A2", "B2", "C3", "D3", "E3", "F3", "G3", "A3", "B3",
        "C4", "D4", "E4", "F4", "G4", "A4", "B4", "C5", "D5", "E5", "F5", "G5", "A5", "B5",
        "C6", "D6", "D6"
    ]

    @staticmethod
    def freq_to_pitch(value):
        nearest_index = PitchConverter.search_nearest(value)
        if 0 <= nearest_index < len(PitchConverter.pitch_string):
            return PitchConverter.pitch_string[nearest_index]
        return PitchConverter.pitch_string[0]

    @staticmethod
    def search_nearest(value):
        pos = bisect.bisect_left(PitchConverter.freq_level, value)
        if pos == 0:
            return 0
        if pos == len(PitchConverter.freq_level):
            return len(PitchConverter.freq_level) - 1

        before = PitchConverter.freq_level[pos - 1]
        after = PitchConverter.freq_level[pos]

        if after - value < value - before:
            return pos
        return pos - 1

In [38]:
def update_rating_by_pitch_coverage(row, user_data):
    coverage = 0.0
    
    highestPitch = PitchConverter.freq_to_pitch(user_data['highest_pitch'])
    lowestPitch = PitchConverter.freq_to_pitch(user_data['lowest_pitch'])
    
    lowest_pitch_met = False
    for pitch, pct in row['pitch_stat'].items():
        if lowestPitch == pitch:
            lowest_pitch_met = True
            coverage += pct
        elif highestPitch == pitch:
            coverage += pct
            break
        elif lowest_pitch_met:
            coverage += pct
    
    row['pred'] += coverage
    return row
        
recommendations_data = recommendations_data.apply(update_rating_by_pitch_coverage, axis=1, user_data=user_data)
recommendations_data.head()

,title,pred,music_id,genre,lyrics,release_date,likes,view,highest_pitch,lowest_pitch,high_pitch_ratio,low_pitch_ratio,pitch_average,pitch_stat,group_id,group_name,group_type,gender
0,"Beauty and the Beast (from ""Beauty and the Bea...",3.662632,395,OST,None,1992-03-30,0,2,882.665004,170.163455,0.669540,0.000000,404.741144,"{'C2': 0.0, 'D2': 0.0, 'E2': 0.0, 'F2': 0.0, '...",72,"Peabo Bryson, Celine Dion",GROUP,MIX
1,Boys And Girls (Feat. Babylon),5.152024,397,RAP,Don't worry about money\n놀자 오빠가 무리할게\n넌 그냥 옆에서...,2015-11-03,0,1,469.292125,123.944968,0.123616,0.000000,270.923070,"{'C2': 0.0, 'D2': 0.0, 'E2': 0.0, 'F2': 0.0, '...",75,지코 (ZICO),SOLO,MALE
2,Don't Stop Me Now,4.174039,396,ROCK,"Tonight, I'm gonna have myself a real good tim...",1976-01-26,0,1,507.861756,164.907098,0.505030,0.000000,319.656593,"{'C2': 0.0, 'D2': 0.0, 'E2': 0.0, 'F2': 0.0, '...",74,Queen,GROUP,MALE
3,Let's Get Loud,3.810624,387,POP,let's get loud let's get loud\nturn the music ...,1999-06-29,0,16,597.670991,205.814429,0.039370,0.226378,346.274942,"{'C2': 0.0, 'D2': 0.0, 'E2': 0.0, 'F2': 0.0, '...",64,Jennifer Lopez,SOLO,FEMALE
4,My All,3.628310,393,RNB,I am thinking of you\n당신을 생각해요\nin my sleeples...,1997-09-16,0,1,694.707279,144.864262,0.173242,0.418525,371.087818,"{'C2': 0.0, 'D2': 0.0, 'E2': 0.0, 'F2': 0.0, '...",69,Mariah Carey,SOLO,FEMALE


In [39]:
recommendations_data.sort_values(by='pred', ascending=False)

,title,pred,music_id,genre,lyrics,release_date,likes,view,highest_pitch,lowest_pitch,high_pitch_ratio,low_pitch_ratio,pitch_average,pitch_stat,group_id,group_name,group_type,gender
1,Boys And Girls (Feat. Babylon),5.152024,397,RAP,Don't worry about money\n놀자 오빠가 무리할게\n넌 그냥 옆에서...,2015-11-03,0,1,469.292125,123.944968,0.123616,0.000000,270.923070,"{'C2': 0.0, 'D2': 0.0, 'E2': 0.0, 'F2': 0.0, '...",75,지코 (ZICO),SOLO,MALE
37,손오공,5.100001,196,DANCE,땅을 보고 계속 올랐지 정상까지\n많은 시련은 보란 듯이 I Always Win\n...,2023-04-24,0,8,364.594405,132.934857,0.246493,0.000000,270.714602,"{'C2': 0.0, 'D2': 0.0, 'E2': 0.0, 'F2': 0.0, '...",47,세븐틴 (SEVENTEEN),GROUP,MALE
8,다행이다,4.989236,383,BALLADE,그대를 만나고\n그대의 머릿결을 만질 수가 있어서\n그대를 만나고\n그대와 마주보며...,2007-04-19,0,22,378.239188,125.874867,0.077957,0.000000,261.205990,"{'C2': 0.0, 'D2': 0.0, 'E2': 0.0, 'F2': 0.0, '...",61,이적,SOLO,MALE
9,사랑은 은하수 다방에서,4.817276,386,FOLK,사랑은 은하수 다방 문 앞에서 만나\n홍차와 냉커피를 마시며\n매일 똑같은 노래를 ...,2011-02-10,0,38,419.766686,179.586040,0.170213,0.000000,287.265146,"{'C2': 0.0, 'D2': 0.0, 'E2': 0.0, 'F2': 0.0, '...",52,10CM,SOLO,MALE
10,소주 한 잔,4.815606,384,BALLADE,술이 한 잔 생각나는 밤\n같이 있는 것 같아요\n그 좋았던 시절들\n이젠 모두 한...,2003-06-05,0,28,883.534154,117.084343,0.149086,0.000000,254.346880,"{'C2': 0.0, 'D2': 0.0, 'E2': 0.0, 'F2': 0.0, '...",62,임창정,SOLO,MALE
15,취중진담,4.783284,180,BALLADE,그래 난 취했는지도 몰라\n실수인지도 몰라\n아침이면 까마득히 생각이 안나\n불안해...,2007-03-15,0,8,688.456088,92.717461,0.236074,0.002653,254.914448,"{'C2': 0.0, 'D2': 0.0, 'E2': 0.0, 'F2': 0.0, '...",22,김동률,SOLO,MALE
16,친구 얘긴데 (feat. Skinny Brown),4.755337,204,INDIE,저기 말야\n나 말고 아는 친구 얘긴데\n별 뜻은 아냐\n그저 네 생각이 좀 궁금해...,2023-06-01,0,7,411.220547,93.726779,0.039872,0.007974,256.574423,"{'C2': 0.0, 'D2': 0.0, 'E2': 0.0, 'F2': 0.0, '...",55,그리즐리 (Grizzly),SOLO,MALE
20,어땠을까 (Feat. 박정현),4.722094,173,RAP,내가 그때 널 잡았더라면\n너와 나 지금보다 행복했을까\n마지막에 널\n안아줬다면 ...,2012-07-15,1,9,620.806132,123.754295,0.485465,0.000000,317.807284,"{'C2': 0.0, 'D2': 0.0, 'E2': 0.0, 'F2': 0.0, '...",15,싸이 (PSY),SOLO,MALE
41,"모든 날, 모든 순간",4.700001,200,OST,네가 없이 웃을 수 있을까\n생각만 해도 눈물이나\n힘든 시간 날 지켜준 사람\n이...,2018-03-20,0,6,348.985553,169.953506,0.027027,0.000000,241.841471,"{'C2': 0.0, 'D2': 0.0, 'E2': 0.0, 'F2': 0.0, '...",51,폴킴,SOLO,MALE
29,응급실,4.699677,186,BALLADE,후회 하고 있어요\n우리 다투던 그 날\n괜한 자존심 때문에\n끝내자고 말을 해버린...,2005-01-20,0,9,447.480460,98.209465,0.184466,0.001618,251.204317,"{'C2': 0.0, 'D2': 0.0, 'E2': 0.0, 'F2': 0.0, '...",36,izi,GROUP,MALE


# 최종 버전: 한 유저에 대해 추천을 하는 함수

In [40]:
from datetime import datetime

def get_recommendations(user_id, df_svd_preds, music_data):
    # 전체 회원을 상대로 로그 기반으로 예측한 평점에서 user_id번 사용자 데이터 추출
    user_predictions = df_svd_preds.iloc[user_index_dict[user_id]]

    # 사용자가 북마크한 음악 조회
    bookmark_data = fetch_bookmark(user_id)
    bookmark_data.columns = ['music_id', 'title']
    
    # 사용자가 북마크한 곡은 추천에서 제외
    recommendations = user_predictions.drop(bookmark_data.title.tolist())
    
    # DataFrame 변환
    recommendations_data = pd.DataFrame(recommendations)
    recommendations_data.columns = ['pred']
    # 추천 결과에 기존 음악 정보 추가
    recommendations_data = recommendations_data.merge(music_data, on='title')
    
    # 사용자 정보 조회
    user_data = fetch_user(user_id)
    
    # 사용자의 선호 장르와 나이대에 따른 가산점 계산
    recommendations_data = recommendations_data.apply(update_rating_by_user_data, axis=1, user_data=user_data)
    
    # 음역대에 따른 가산점 계산
    recommendations_data = recommendations_data.apply(update_rating_by_pitch_coverage, axis=1, user_data=user_data)
    
    # 예측 평점이 높은 순으로 정렬
    recommendations_data.sort_values(by='pred', ascending=False, inplace=True)
    recommendations_data.reset_index(drop=True, inplace=True)
    
    # 상위 100개만 선택
    recommendations_data = recommendations_data[:100]
    recommendations_data = recommendations_data[['music_id', 'pred']]
    
    # 추천 시점 저장
    now = datetime.now()
    version = now.strftime("%Y-%m-%d %H:%M:%S")
    
    save_recom_musics(user_id, recommendations_data, version)
    

In [41]:
user_index_dict.keys()

dict_keys([1, 2, 5, 6, 8, 19, 21])

In [43]:
[(user_id, ) for user_id in user_index_dict.keys()]

[1, 2, 5, 6, 8, 19, 21]